# Introduction

Suite à la publication de 340 notices d'entités LRM par le groupe Systèmes et Données https://zenodo.org/record/7358799 voici un ensemble de scripts pour visualiser un peu leur contenu

Les données diffusées en iso2709 ont été converties en MarcXML pour faciliter les traitements.

4 fichiers sont fournis :

In [5]:
manifs_filename = "UMB_Manifestations.xml"
oeuvres_expressions_filename = "UMA_Oeuvres_Expressions.xml"
autres_entites_filename = "UMA_Autres_Entites_Liees.xml"
items_filename = "UMH_Items.xml"

In [20]:
import SRUextraction as sru # import du fichier https://github.com/Lully/bnf-sru/blob/master/SRUextraction.py
from lxml import etree
from string import ascii_lowercase

ns = {"marc": "http://www.loc.gov/MARC21/slim"}

In [7]:
manifs_file = etree.parse(manifs_filename)
oeuvres_expressions_file = etree.parse(oeuvres_expressions_filename)
autres_entites_file = etree.parse(autres_entites_filename)
items_file = etree.parse(items_filename)

### Fonctions d'extraction

In [18]:
class Record:
    def __init__(self, xml_record, rectype):
        self.xml = xml_record
        self.init_type = rectype
        self.id = sru.record2fieldvalue(self.xml, "001")
        self.txt = sru.xml2seq(self.xml)
        self.type = get_type(xml_record, rectype)
        self.label = get_label(self)
    
    def __repr__(self):
        return etree.tostring(self.xml).decode("utf-8")


class Manifestation(Record):
    def __init__(self, xml_record, rectype):
        super().__init__(xml_record, rectype)
        self.toExpression = manif2expression(self.xml)
        pass


class Oeuvre(Record):
    def __init__(self, xml_record, rectype):
        pass
    
class Expression(Record):
    def __init__(self, xml_record, rectype):
        pass

def manif2expression(xml_record):
    tags_2expression = ["507", "577"]
    dict_expressions = {}
    for tag in tags_2expression:
        for field_occ in xml_record.xpath(f"*[@tag='{tag}']"):
            recordid = sru.field2subfield(field_occ, "3")
            accesspoint = accesspoint2label(field_occ)
            dict_expressions[recordid] = accesspoint
    return dict_expressions

    
def accesspoint2label(field):
    # Construction du point d'accès sans $
    label = []
    for subf in field.xpath("*[@code]"):
        code = subf.get("code")
        if code in ascii_lowercase and subf.text:
            label.append(subf.text)
    label = ", ".join(label)
    return label
    
def get_type(xml_record, rectype):
    entity_type = None
    if rectype in "mi":
        entity_type = rectype
    elif rectype in "oe":
        equiv = {"a": "o", "b": "e", "x": "x"}
        f154a = sru.record2fieldvalue(xml_record, "154$a")
        if f154a and f154a[0] in equiv:
            entity_type = equiv[f154a[0]]
        else:
            entity_type = "z"
    else:
        entity_type = rectype
    if entity_type is None:
        print(etree.tostring(xml_record))
    return entity_type
            
        
def get_label(record):
    label = []
    if record.type in "mipc":
        label.append(sru.record2fieldvalue(record.xml, "200$a"))
    elif record.type in "eox":
        for field in record.xml.xpath("*[@tag]"):
            tag = field.get("tag")
            if tag.startswith("2"):
                if sru.field2subfield(field, "a"):
                    label.append(sru.field2subfield(field, "a"))
                else:
                    label.append("*"*20)
                    label.append("$a vide")
                    label.append(sru.field2value(field))
    return ", ".join(label)

# Relations entre entités

Liens : 531 $3

Point d'accès structuré: zone 511

Note (non exploitable informatiquement) 370

# Les manifestations

Besoins : 
* pour une manifestation, récupérer les métas principales :
  * titre, point d'accès, auteur
  * liens aux entités Expressions (et oeuvre ?)
* pour une manif, récupérer la liste des expressions et la liste des oeuvres (lien direct)
  * créer des attributs à la manifs, en fournissant l'expression source pour chaque oeuvre identifié


In [21]:
manifs = [Manifestation(manif, "m") for manif in manifs_file.xpath(".//marc:record", namespaces=ns)]
oeuvres_expr = [Record(oe, "oe") for oe in oeuvres_expressions_file.xpath(".//marc:record", namespaces=ns)]  

## Liens dans les manifs

* 507 : point d'accès autorisé Titre vers l'expression
* 577 : point d'accès autorisé Auteur-Titre vers l'expression

Chaque manifestation a un attribut .toExpression qui est un dictionnaire :
 * clé : numéro de notice d'expression
 * valeur : point d'accès (sans les dollars)

In [22]:
for manif in manifs:
    for expr in manif.toExpression:
        print(manif.id, expr, manif.toExpression[expr])

UMLRM0003 ULRM0002 Nosferatu, eine Symphonie des Grauens, film, Version restaurée, 60mn, 1995, Français
UMLRM0006 UMLRM0005 Stoker, Bram (1847-1912), Dracula, Anglais, Texte noté
UMLRM0009 UMLRM0008 Nosferatu, Phantom der Nacht, film, français
UMLRM0012 UMLRM0011 Palma, Paola (1971-….), Vezyroglou, Dimitri (1970-….), "Nosferatu le vampire", Français, Texte noté
UMLRM0015  UMLRM0015 Jean-Claude Gallotta, Nosferatu, ballet de l'Opéra, programme, 2006-05, Texte noté. Image fixe
UMLRM0018 UMLRM0005 Stoker, Bram (1847-1912), Dracula, Anglais, Texte noté
UMLRM0021 UMLRM0020 Stoker, Bram (1847-1912), Dracula, Français, Molitor, Texte noté
UMLRM0024 UMLRM0020 Stoker, Bram (1847-1912), Dracula, Français, Molitor, Texte noté
UMLRM0027 UMLRM0020  Stoker, Bram (1847-1912), Dracula, Français, Molitor, Texte noté
UMLRM0030 UMLRM0020 Stoker, Bram (1847-1912), Dracula, Français, Molitor, Texte noté
UMLRM0033 UMLRM0032 Stoker, Bram (1847-1912), Dracula, Choix, Bruneel-Ziane, Français, Molitor, Texte n

## Liste des ID et titres

In [8]:
for el in manifs:
    print(el.id, el.label)

UMLRM0003 Nosferatu
UMLRM0006 Dracula
UMLRM0009  Nosferatu, fantôme de la nuit
UMLRM0012 "Nosferatu le vampire"
UMLRM0015 Jean-Claude Gallotta, Nosferatu, ballet de l'Opéra
UMLRM0018 Dracula
UMLRM0021  Dracula
UMLRM0024 Dracula
UMLRM0027 Dracula
UMLRM0030 Dracula
UMLRM0033 Dracula
UMLRM0036 Dracula
UMLRM0039 Dracula
UMLRM0042  Dracula
UMLRM0045 Dracula
UMLRM0048 Dracula
UMLRM0051 Dracula
UMLRM0054 Dracula
UMLRM0057 Dracula
UMLRM0060  Dracula
UMLRM0063 Dracula
UMLRM0066 Drácula
UMLRM0069 Dracula‎
UMLRM0072 Bram Stoker
UMLRM0077 Dracula¤suivi de L'invité de Dracula
UMLRM0080 Bram Stoker's Dracula
UMLRM0083  "Dracula"
UMLRM0086 Dracula
UMLRM0089 Le vampire au fil des siècles
UMLRM0092 Nosferatu
UMLRM0095 Nosferatu
UMLRM0101 Nosferatu
UMLRM0104 Nosferatu
UMLRM0107 Dracula has risen from the grave‎
UMLRM0110 Dracula
UMLRM0113  Bram Stoker’s Dracula
UMLRM0116 Dracula l'immortel
UMLRM0119 Dracula l'immortel
UMLRM0122  Dracula, mort et heureux de l’être
UMLRM0128 Bram Stoker’s notes for “Drac

# Oeuvres expressions

Liste des titres

## Distinguer oeuvres et expressions

Label pos.9 permet de distinguer oeuvres et expressions : valeurs h / t (auteur-titre, ou titre seul) 

Zone 154 $a pos1 : a = oeuvre ; b = expression ; x = ne s'applique pas

Point d'accès
* 231 pour les oeuvres
* 232 pour les expressions

In [9]:
for el in oeuvres_expr:
    print(el.id, el.label)

UMLRM0001 ********************, $a vide, $7 ba0yba0y $8 freger $t Nosferatu, eine Symphonie des Grauens $c film
UMLRM0002 Nosferatu, eine Symphonie des Grauens
UMLRM0004 Stoker, Bram (1847-1912)
UMLRM0005 Stoker, Bram (1847-1912)
UMLRM0007  Nosferatu, Phantom der Nacht
UMLRM0008 Nosferatu, Phantom der Nacht
UMLRM0010 Palma, Paola (1971-….)¤Vezyroglou, Dimitri (1970-….)
UMLRM0011 Palma, Paola (1971-….)¤Vezyroglou, Dimitri (1970-….)
UMLRM0013 Jean-Claude Gallotta, Nosferatu, ballet de l'Opéra
UMLRM0014 Jean-Claude Gallotta, Nosferatu, ballet de l'Opéra
UMLRM0020 Stoker, Bram (1847-1912)
UMLRM0031 Stoker, Bram (1847-1912)
UMLRM0032 Stoker, Bram (1847-1912)
UMLRM0034 Humphries, Tudor (1953-….)
UMLRM0035 Humphries, Tudor (1953-….)
UMLRM0037 Stoker, Bram (1847-1912)
UMLRM0038 Stoker, Bram (1847-1912)
UMLRM0041 Stoker, Bram (1847-1912)
UMLRM0047 Stoker, Bram (1847-1912)
UMLRM0050 Stoker, Bram (1847-1912)
UMLRM0053 Stoker, Bram (1847-1912)
UMLRM0056 Stoker, Bram (1847-1912)
UMLRM0059 Stoker, B

# Coquilles rencontrées

https://semestriel.framapad.org/p/bugs_unimarclrm